1. Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

In [59]:
!pip install -q pymorphy2
!pip install -q stop_words

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import lru_cache
from collections import Counter
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from torch.utils.data import DataLoader, Dataset
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from tqdm import tqdm

# Dataset loading

In [61]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [62]:
df_train.head(3)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0


In [63]:
df_test.head(3)

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."


In [64]:
df_val.head(3)

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0


# Preprocessing

In [65]:
sw = set(get_stop_words("ru"))
puncts = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [66]:
%%time

tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)
df_test['text'] = df_test['text'].progress_apply(preprocess_text)

100%|██████████| 22684/22684 [00:26<00:00, 848.78it/s]

CPU times: user 4min 20s, sys: 1.52 s, total: 4min 22s
Wall time: 4min 38s


In [67]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [70]:
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [94]:
max_words = 2000
max_len = 20
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [73]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [74]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [75]:
tokens_filtered_top[:20]

['rt',
 'd',
 'хотеть',
 'знать',
 'ян',
 'мочь',
 'новый',
 'любить',
 'завтра',
 'мой',
 'хороший',
 'делать',
 'день',
 'смотреть',
 'вообще',
 'думать',
 'идти',
 'иня',
 'блин',
 'самый']

In [76]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [77]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen-len(result))
    return padding + result[-maxlen:]

In [78]:
%%time

x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

CPU times: user 22.5 s, sys: 78.1 ms, total: 22.6 s
Wall time: 22.7 s


In [79]:
x_train.shape

(181467, 20)

In [80]:
x_train[13]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 125,  20,  11,  27], dtype=int32)

In [82]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [83]:
train_dataset = DataWrapper(x_train, df_train['class'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['class'].values)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Recurrent Neural Network (RNN)

In [84]:
class RNNFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True) :
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        rnn_out, ht = self.rnn(x)
        # rnn_out: тензор с выходными фичами с полследнего слоя для каждого t
        # ht: тензор с полседними скрытыми состояниями по слоям

        if self.use_last:
          last_tensor = rnn_out[:,-1,:]
        else:
          #use mean
          last_tensor = torch.mean(rnn_out[:,:], dim=1)

        out = self.linear(last_tensor)

        return torch.sigmoid(out)

In [85]:
rnn_init = RNNFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(rnn_init.parameters(), lr=10e-3)
criterion = nn.BCELoss()

In [86]:
print(rnn_init)
print("Parameters:", sum([param.nelement() for param in rnn_init.parameters()]))

RNNFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (rnn): RNN(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 259861


In [99]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [88]:
rnn_init.to(device)

RNNFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (rnn): RNN(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [97]:
rnn_init = rnn_init.to(device)
rnn_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in tqdm(range(epochs)):
    rnn_init.train()
    running_items, running_right = 0.0, 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = rnn_init(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчёт ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчёт метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

    # выводим статистику в процессе обучения
    rnn_init.eval()
    print(f'Epoch [{epoch + 1} / {epochs}]. ' \
          f'Step [{i + 1} / {len(train_loader)}]. ' \
          f'Loss {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_rigth, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = rnn_init(data[0].to(device))

        # подсчёт ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчёт метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_rigth += (test_labels == pred_test_labels).sum()

    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_rigth / test_running_total:.3f}')

print('Training is finished!')


 20%|██        | 1/5 [00:02<00:10,  2.52s/it]

Epoch [1 / 5]. Step [355 / 355]. Loss 0.536. Acc: 0.711. Test loss: 0.518. Test acc: 0.695


 40%|████      | 2/5 [00:05<00:08,  2.84s/it]

Epoch [2 / 5]. Step [355 / 355]. Loss 0.500. Acc: 0.712. Test loss: 0.588. Test acc: 0.695


 60%|██████    | 3/5 [00:08<00:05,  2.81s/it]

Epoch [3 / 5]. Step [355 / 355]. Loss 0.547. Acc: 0.711. Test loss: 0.619. Test acc: 0.696


 80%|████████  | 4/5 [00:10<00:02,  2.60s/it]

Epoch [4 / 5]. Step [355 / 355]. Loss 0.547. Acc: 0.712. Test loss: 0.625. Test acc: 0.700


100%|██████████| 5/5 [00:13<00:00,  2.65s/it]

Epoch [5 / 5]. Step [355 / 355]. Loss 0.521. Acc: 0.712. Test loss: 0.548. Test acc: 0.696
Training is finished!


# Long Short-Term Memory (LSTM)

In [100]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        lstm_out, ht = self.lstm(x)
        # lstm_out: тензор с выходными фичами с полследнего слоя для каждого t
        # ht: тензор с полседними скрытыми состояниями по слоям

        if self.use_last:
          last_tensor = lstm_out[:,-1,:]
        else:
          #use mean
          last_tensor = torch.mean(lstm_out[:,:], dim=1)

        out = self.linear(last_tensor)

        return torch.sigmoid(out)

In [101]:
lstm_init = LSTMFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=10e-3)
criterion = nn.BCELoss()

In [102]:
print(lstm_init)
print("Parameters:", sum([param.nelement() for param in lstm_init.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (lstm): LSTM(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 271381


In [104]:
lstm_init = lstm_init.to(device)
lstm_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in tqdm(range(epochs)):
    lstm_init.train()
    running_items, running_right = 0.0, 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = lstm_init(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчёт ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчёт метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

    # выводим статистику в процессе обучения
    lstm_init.eval()
    print(f'Epoch [{epoch + 1} / {epochs}]. ' \
          f'Step [{i + 1} / {len(train_loader)}]. ' \
          f'Loss {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_rigth, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = lstm_init(data[0].to(device))

        # подсчёт ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчёт метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_rigth += (test_labels == pred_test_labels).sum()

    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_rigth / test_running_total:.3f}')

print('Training is finished!')

 20%|██        | 1/5 [00:02<00:10,  2.71s/it]

Epoch [1 / 5]. Step [355 / 355]. Loss 0.560. Acc: 0.722. Test loss: 0.505. Test acc: 0.699


 40%|████      | 2/5 [00:05<00:07,  2.54s/it]

Epoch [2 / 5]. Step [355 / 355]. Loss 0.510. Acc: 0.724. Test loss: 0.513. Test acc: 0.701


 60%|██████    | 3/5 [00:08<00:06,  3.00s/it]

Epoch [3 / 5]. Step [355 / 355]. Loss 0.505. Acc: 0.728. Test loss: 0.580. Test acc: 0.701


 80%|████████  | 4/5 [00:11<00:02,  2.78s/it]

Epoch [4 / 5]. Step [355 / 355]. Loss 0.443. Acc: 0.730. Test loss: 0.536. Test acc: 0.701


100%|██████████| 5/5 [00:13<00:00,  2.78s/it]

Epoch [5 / 5]. Step [355 / 355]. Loss 0.477. Acc: 0.732. Test loss: 0.626. Test acc: 0.700
Training is finished!


# Gated Recurrent Unit (GRU)

In [135]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        gru_out, ht = self.gru(x)
        # lstm_out: тензор с выходными фичами с полследнего слоя для каждого t
        # ht: тензор с полседними скрытыми состояниями по слоям

        if self.use_last:
          last_tensor = gru_out[:,-1,:]
        else:
          #use mean
          last_tensor = torch.mean(gru_out[:,:], dim=1)

        out = self.linear(last_tensor)

        return torch.sigmoid(out)

In [136]:
gru_init = GRUFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(gru_init.parameters(), lr=10e-3)
criterion = nn.BCELoss()

In [137]:
print(gru_init)
print("Parameters:", sum([param.nelement() for param in gru_init.parameters()]))

GRUFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (gru): GRU(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 267541


In [139]:
gru_init = gru_init.to(device)
gru_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in tqdm(range(epochs)):
    gru_init.train()
    running_items, running_right = 0.0, 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = gru_init(inputs)

        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчёт ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчёт метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()

    # выводим статистику в процессе обучения
    gru_init.eval()
    print(f'Epoch [{epoch + 1} / {epochs}]. ' \
          f'Step [{i + 1} / {len(train_loader)}]. ' \
          f'Loss {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_rigth, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = gru_init(data[0].to(device))

        # подсчёт ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчёт метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_rigth += (test_labels == pred_test_labels).sum()

    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_rigth / test_running_total:.3f}')

print('Training is finished!')

 20%|██        | 1/5 [00:03<00:12,  3.19s/it]

Epoch [1 / 5]. Step [355 / 355]. Loss 0.487. Acc: 0.717. Test loss: 0.539. Test acc: 0.699


 40%|████      | 2/5 [00:05<00:08,  2.88s/it]

Epoch [2 / 5]. Step [355 / 355]. Loss 0.534. Acc: 0.719. Test loss: 0.543. Test acc: 0.703
Epoch [3 / 5]. Step [355 / 355]. Loss 0.564. Acc: 0.720. 

 60%|██████    | 3/5 [00:09<00:06,  3.00s/it]

Test loss: 0.545. Test acc: 0.704
Epoch [4 / 5]. Step [355 / 355]. Loss 0.530. Acc: 0.722. 

 80%|████████  | 4/5 [00:12<00:03,  3.07s/it]

Test loss: 0.575. Test acc: 0.700


100%|██████████| 5/5 [00:15<00:00,  3.08s/it]

Epoch [5 / 5]. Step [355 / 355]. Loss 0.553. Acc: 0.722. Test loss: 0.532. Test acc: 0.703
Training is finished!
